In [7]:
import torch
import os

print(f"PyTorch Version: {torch.__version__}")
print(f"Current Directory: {os.getcwd()}")
print(f"Files in Directory: {os.listdir('.')}")


PyTorch Version: 2.0.1+cu117
Current Directory: /app
Files in Directory: ['.git', '.gitignore', '.idea', '.ipynb_checkpoints', 'docker-compose.yml', 'Dockerfile', 'LICENSE', 'main.py', 'nb.ipynb', 'README.md', 'requirements.txt']


In [6]:
print(torch.cuda.is_available()) 

False
